In [1]:
from dataset_preprocessing import ProductionModeDataset

import torch
from sklearn.inspection import permutation_importance
import numpy as np
import uproot
import torch.nn as nn
from matplotlib import pyplot as plt

from Classifier_module import Classifier
from torch.autograd import Variable


In [24]:
################# THIS IS WHERE YOU CHOOSE WHAT TO LOAD ################################
path_to_model = "./models/twoLayerModel_gpu_emu0"

event_type = "emu"  # could be ee, emu, mumu
root_path = "/depot-new/cms/top/mcnama20/TopSpinCorr-Run2-Entanglement/CMSSW_10_2_22/src/TopAnalysis/Configuration/analysis/diLeptonic/three_files/Nominal"

file = root_path + "/" + event_type + "_modified_root_1.root"

In [25]:
# model = 0
model = Classifier()
model.load_state_dict(torch.load(path_to_model))
model.eval()

Classifier(
  (model): Sequential(
    (0): Linear(in_features=80, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=256, out_features=3, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
  )
)

In [26]:



data_object = ProductionModeDataset(file)
x = data_object.get_eval_data()


['lb_delta_eta', 'lbbar_delta_eta', 'lnu_delta_eta', 'lnubar_delta_eta', 'lbarb_delta_eta', 'lbarbbar_delta_eta', 'lbarnu_delta_eta', 'lbarnubar_delta_eta', 'bnu_delta_eta', 'bnubar_delta_eta', 'bbarnu_delta_eta', 'bbarnubar_delta_eta', 'lb_delta_phi', 'lbbar_delta_phi', 'lnu_delta_phi', 'lnubar_delta_phi', 'lbarb_delta_phi', 'lbarbbar_delta_phi', 'lbarnu_delta_phi', 'lbarnubar_delta_phi', 'bnu_delta_phi', 'bnubar_delta_phi', 'bbarnu_delta_phi', 'bbarnubar_delta_phi', 'wplusb_delta_eta', 'wplusbbar_delta_eta', 'wminusb_delta_eta', 'wminusbbar_delta_eta', 'wplusb_delta_phi', 'wplusbbar_delta_phi', 'wminusb_delta_phi', 'wminusbbar_delta_phi', 'top_eta', 'top_boosted_eta', 'tbar_eta', 'tbar_boosted_eta', 'ttbar_delta_eta', 'ttbar_eta', 'llbar_delta_eta', 'bbbar_delta_eta', 'nunubar_delta_eta', 'top_phi', 'tbar_phi', 'ttbar_phi', 'ttbar_delta_phi', 'llbar_phi', 'llbar_delta_phi', 'bbbar_phi', 'bbbar_delta_phi', 'nunubar_phi', 'nunubar_delta_phi', 'l_eta', 'lbar_eta', 'l_phi', 'lbar_phi', '

In [33]:
weight = x[:,81]
target = x[:,80]
target = Variable(torch.from_numpy(target).type(torch.LongTensor))
y = np.transpose(x)
y = np.delete(y, [80, 81, 82], 0)
y = np.transpose(y)
input = Variable(torch.from_numpy(y).type(torch.Tensor))

In [32]:
model(input)[0,:]

tensor([-0.0995,  2.2617, -0.1250], grad_fn=<SliceBackward>)

In [29]:
target[0]

tensor(1)

In [30]:
from skorch import NeuralNetClassifier
from sklearn.inspection import permutation_importance

In [34]:
nn = NeuralNetClassifier(model, max_epochs=0, lr=0.00, batch_size=5000)
nn.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Classifier(
    (model): Sequential(
      (0): Linear(in_features=80, out_features=256, bias=True)
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Linear(in_features=256, out_features=3, bias=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  ),
)

In [35]:
events = uproot.open(file)
key = events.keys()[0]
input_variables = events[key].keys()

In [37]:
r = permutation_importance(nn, input, target, \
                           n_repeats=30, \
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{input_variables[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")


ttbar_pt0.083 +/- 0.002
l_phi   0.060 +/- 0.002
llbar_delta_phi0.060 +/- 0.002
ttbar_phi0.056 +/- 0.002
nu_phi  0.053 +/- 0.002
lbarnu_delta_phi0.053 +/- 0.003
b_phi   0.053 +/- 0.002
wplus_phi0.052 +/- 0.002
tbar_phi0.052 +/- 0.002
lbarb_delta_phi0.051 +/- 0.002
lb_delta_eta0.051 +/- 0.002
l_eta   0.051 +/- 0.002
lbar_eta0.050 +/- 0.002
wminus_phi0.050 +/- 0.002
lbarbbar_delta_eta0.050 +/- 0.003
nubar_phi0.050 +/- 0.002
lnubar_delta_phi0.050 +/- 0.002
llbar_phi0.050 +/- 0.002
bbar_phi0.049 +/- 0.002
bbarnubar_delta_phi0.049 +/- 0.002
bnu_delta_phi0.048 +/- 0.002
wminusbbar_delta_phi0.047 +/- 0.001
wplusb_delta_phi0.047 +/- 0.002
lbar_phi0.046 +/- 0.002
bbarnu_delta_phi0.046 +/- 0.002
llbar_delta_eta0.045 +/- 0.002
bbbar_delta_phi0.045 +/- 0.002
nunubar_delta_phi0.045 +/- 0.002
bbar_eta0.045 +/- 0.002
b_eta   0.045 +/- 0.002
top_phi 0.045 +/- 0.002
lbbar_delta_phi0.044 +/- 0.002
bbbar_phi0.043 +/- 0.002
lbarbbar_delta_phi0.043 +/- 0.002
nunubar_phi0.042 +/- 0.002
wplusbbar_delta_phi0.0

In [36]:
nn.score(input, target)

0.6158876084423152